<a href="https://colab.research.google.com/github/ZeroLeon/NLP_Task_with_FastAI/blob/master/Pre_processing_taptap_comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TapTap Comment Classification

## Data Preprocessing

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive/fastai'
path = Path(base_dir +'/data/taptap_comment')
path.mkdir(parents=True,exist_ok=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Preporcessing

In [0]:
!pip install pandas-profiling

In [0]:
import pandas as pd
import pandas_profiling

In [0]:
tap_df = pd.read_csv(path/'taptap_raw.csv')

In [0]:
tap_df.head()

,web-scraper-order,web-scraper-start-url,start_link,start_link-href,comment_texts,stars
0,1561221864-4198,https://www.taptap.com/category/e381?page=18,艾比之星,https://www.taptap.com/app/78946,看见这个游戏上了日推，于是便心血来潮的的下载玩了一下，现在借助Tap这个平台发一些玩这个游戏...,width: 56px
1,1561218498-1412,https://www.taptap.com/category/e381?page=34,刀剑兵器谱,https://www.taptap.com/app/16404,之前手机玩过一段时间，个人不太喜欢这种类型。很不明白，为什么不做刀剑2，当年端游方面很着迷入...,width: 28px
2,1561219394-3191,https://www.taptap.com/category/e381?page=24,魔窟-无尽的地下城,https://www.taptap.com/app/15386,我想给厂家提个建议：可否出一个云存档，上传玩家的游戏纪录，否则辛辛苦苦肝出来的存档，卸载后就...,width: 56px
3,1561218635-1725,https://www.taptap.com/category/e381?page=32,六十五点零,https://www.taptap.com/app/53320,▶前言:专治手欠手残，不知诸位手好了没有？▶评论标题:你的手由我来守护▶简评:①Tap上又一...,width: 56px
4,1561219685-3856,https://www.taptap.com/category/e381?page=20,地下城堡：炼金术师的魔幻之旅\n CN,https://www.taptap.com/app/11471,《地下城堡：炼金术师的魔幻之旅》2014.10.1【雷霆游戏】淘金互娱【剧情背景】误入荒野天...,width: 56px


In [0]:
tap_df.shape

(3338, 6)

In [0]:
drop_list = tap_df.columns.values
drop_list

array(['web-scraper-order', 'web-scraper-start-url', 'start_link', 'start_link-href', 'comment_texts', 'stars'],
      dtype=object)

In [0]:
drop_list = drop_list[:-3]
drop_list

array(['web-scraper-order', 'web-scraper-start-url', 'start_link'], dtype=object)

In [0]:
tap_df.drop(columns=drop_list,inplace=True)

In [0]:
tap_df.head()

,start_link-href,comment_texts,stars
0,https://www.taptap.com/app/78946,看见这个游戏上了日推，于是便心血来潮的的下载玩了一下，现在借助Tap这个平台发一些玩这个游戏...,width: 56px
1,https://www.taptap.com/app/16404,之前手机玩过一段时间，个人不太喜欢这种类型。很不明白，为什么不做刀剑2，当年端游方面很着迷入...,width: 28px
2,https://www.taptap.com/app/15386,我想给厂家提个建议：可否出一个云存档，上传玩家的游戏纪录，否则辛辛苦苦肝出来的存档，卸载后就...,width: 56px
3,https://www.taptap.com/app/53320,▶前言:专治手欠手残，不知诸位手好了没有？▶评论标题:你的手由我来守护▶简评:①Tap上又一...,width: 56px
4,https://www.taptap.com/app/11471,《地下城堡：炼金术师的魔幻之旅》2014.10.1【雷霆游戏】淘金互娱【剧情背景】误入荒野天...,width: 56px


In [0]:
tap_df.stars.value_counts()

width: 56px    1295
width: 70px    1022
width: 42px     697
width: 28px     181
width: 14px     139
Name: stars, dtype: int64

In [0]:
tap_df.dtypes

start_link-href    object
comment_texts      object
stars              object
dtype: object

In [0]:
#make a function to turn the numbers of px into sentiment polarity
#70px -> 5 stars, 56px -> 4 stars, (42,28,14)px -> less than 3 stars
def px_to_star(px):
  if ('70' in str(px)) or ('56' in str(px)):
    return 1  # means satisfied
  else:
    return 0 # unsatisfied

In [0]:
tap_df['sentiment'] = tap_df.stars.apply(px_to_star)

In [0]:
tap_df.to_csv('taptap_processed.csv',index=False)

## Bad comment processing

In [0]:
df_bad = pd.read_csv(path/'taptap_bad_comment.csv')

In [0]:
df_bad.head()

,web-scraper-order,web-scraper-start-url,stars,review
0,1561390296-4907,https://www.taptap.com/app/85838/review?order=...,width: 14px,这游戏根本就不是正统暗黑！暗金（传说）包括套装才两条随机魔法属性，暗金武器防具全部一个装备技...
1,1561390321-5108,https://www.taptap.com/app/85838/review?order=...,width: 42px,大家好，作为一个不会写长评的用户，请老铁们凑活着看，下面是我肝一天后的体验！首先要说明的是，...
2,1561390272-4706,https://www.taptap.com/app/85838/review?order=...,width: 14px,我也不知道我为啥玩个弓箭手，其它职业我不清楚，攻击键一直按一直爽，移动键一直按一直当你没按！...
3,1561390336-5259,https://www.taptap.com/app/85838/review?order=...,width: 28px,说一说自己玩了五个小时的看法。注：无氪 倾家荡产冲战榜80一个类暗黑手游，抄什么的就不谈了，...
4,1561390289-4871,https://www.taptap.com/app/85838/review?order=...,width: 42px,喜闻拉结尔开服首先发点最新攻略游戏攻略职业：平民游侠，小氪骑士，大氪傀儡，法师/蛮子目前下水...


In [0]:
df_bad.review.iloc[1]

'大家好，作为一个不会写长评的用户，请老铁们凑活着看，下面是我肝一天后的体验！首先要说明的是，一个游戏的好坏，不能仅用氪金与不氪金来衡量，而是对于光大玩家（平民玩家）来说，轻氪或者不氪，这游戏是否还能玩的下去。。。事实证明，拉结尔还是可以稍微玩一下的（稍微），请不要注重排行榜，请不要注重过副本，副本打不动就慢慢磨，总会磨过去的！！！下面是今天公测的游戏体验优点：打击感和下副本刷怪的畅快感让我为之一振，确实很不错，比前几次内测舒服了很多，喜欢暗黑风地下城的小伙伴可以坚持玩下去，毕竟手游里我觉得这已经算是很好了！缺点：挺肝，而且不知道肝了些啥，反正你不打，也没啥可以玩的！！！氪金，各种材料商店都有卖，想要？用钻石买，没有想要的？用钻石刷新！！（怎么感觉和阴阳师的买御魂很像）！！然后就是时装，宠物，这些不多说了，不氪玩个***。总的来说，佛系玩家可以玩一玩，千万别纠结排行榜啥的，看着爽爽就行了。'

In [0]:
df_bad.dropna(inplace=True)

In [0]:
df_bad.shape

(3453, 4)

In [0]:
min_len = 30
max_len = 500

In [0]:
df_bad_new = df_bad[  (min_len < df_bad.review.str.len()) & (df_bad.review.str.len() < max_len)  ].copy()

In [0]:
df_bad_new.reset_index(drop=True,inplace=True)

In [0]:
df_bad_new.tail(20)

,web-scraper-order,web-scraper-start-url,stars,review
1727,1561392138-7870,https://www.taptap.com/app/134490/review?order...,width: 14px,我的世界生存战争，这和我的世界有什么关系，又和生存战争有什么关系，这个名字呀，因为有我的世界...
1728,1561392125-7752,https://www.taptap.com/app/134490/review?order...,width: 14px,等等，这就是传说中的名字致敬我的世界，玩法借鉴饥荒的游戏吗，秀啊
1729,1561392157-8059,https://www.taptap.com/app/134490/review?order...,width: 70px,这个是我玩过最好的游戏，首先使我感到震惊的是他的游戏名称，(我的世界生存战争)，啊，多么山寨...
1730,1561392174-8144,https://www.taptap.com/app/134490/review?order...,width: 14px,这个东西，一次蹭3个游戏热度，一个mc，一个生存战争，一个饥荒这3个我都玩过，我不知道这样连...
1731,1561392153-8016,https://www.taptap.com/app/134490/review?order...,width: 14px,名字.....无语厂商真是懒到一定境界了画面，这是饥荒吗?!而且画的连饥荒都不如.........
1732,1561392159-8075,https://www.taptap.com/app/134490/review?order...,width: 14px,当我看到这款游戏的时候，我感觉十分好奇，就点了进来，进来一看，我滴个乖乖！这是个啥？把两大不...
1733,1561392136-7845,https://www.taptap.com/app/134490/review?order...,width: 14px,迷你世界：…………………………我去你的，你抄我的干嘛？我火呀？我的世界：迷你世界抄袭我也就算...
1734,1561392067-7298,https://www.taptap.com/app/134490/review?order...,width: 28px,挖槽，结合我的世界，生存战争，饥荒的绝世神作居然在这里被窝找到了，哇 哇哇 哇哇哇哇哇哇...
1735,1561392125-7740,https://www.taptap.com/app/134490/review?order...,width: 14px,真厉害啊，不紧抄我的世界，还抄迷你世界，真厉害啊，这种游戏还出！！🤬一个星都不想给
1736,1561392107-7644,https://www.taptap.com/app/134490/review?order...,width: 14px,好啊还是盗版游戏，吃鸡、我的世界、迷你世界，都抄袭了，怪牛逼啊！


In [0]:
def px_to_star(px):
  if ('70' in str(px)) or ('56' in str(px)):
    return 1  # means satisfied
  else:
    return 0 # unsatisfied

In [0]:
df_bad_new['sentiment'] = df_bad_new.stars.apply(px_to_star)

In [0]:
df_bad_new.tail()

,web-scraper-order,web-scraper-start-url,stars,review,sentiment
1742,1561392125-7750,https://www.taptap.com/app/134490/review?order...,width: 14px,又是个抄袭饥荒的，我就奇怪了这次为什么不是网易和4399抄了？,0
1743,1561392079-7420,https://www.taptap.com/app/134490/review?order...,width: 14px,我们才能在未来之路上渐行渐远………emmmmmm渐行渐远的意思百度一下,0
1744,1561392090-7516,https://www.taptap.com/app/134490/review?order...,width: 42px,"Emmmm,我也是醉了。我想问，背景是迷你世界的吗。貌似见过，真棒👍。抄不抄袭我不说了，因为...",0
1745,1561392121-7713,https://www.taptap.com/app/134490/review?order...,width: 14px,游戏盈利我可以理解，但是这游戏真是满身铜臭，怎么看怎么像骗不懂事的小孩坑爸妈钱的游戏,0
1746,1561392131-7803,https://www.taptap.com/app/134490/review?order...,width: 14px,看见比mini评分还低我就放心了，看见这人物我第一个想到mini，看了一眼，居然都是深圳的，...,0


In [0]:
df_good = pd.read_csv(path/'taptap_processed.csv',index_col=0)

In [0]:
df_good.shape

(3334, 4)

In [0]:
df_good[  (30 < df_good.comment_texts.str.len()) & (df_good.comment_texts.str.len() < 2500)  ].shape

(3134, 4)

In [0]:
df_good_new = df_good[  (30 < df_good.comment_texts.str.len())].copy()

In [0]:
df_good_new.reset_index(drop=True,inplace=True)

In [0]:
df_good_new.drop(columns=['start_link-href','stars'],inplace=True)

In [0]:
df_bad_new.drop(columns=['web-scraper-order','web-scraper-start-url','stars'],inplace=True)

In [0]:
df_good_new.columns = ['review', 'sentiment']

In [0]:
df_good_new.head()

,review,sentiment
0,看见这个游戏上了日推，于是便心血来潮的的下载玩了一下，现在借助Tap这个平台发一些玩这个游戏...,1
1,之前手机玩过一段时间，个人不太喜欢这种类型。很不明白，为什么不做刀剑2，当年端游方面很着迷入...,0
2,我想给厂家提个建议：可否出一个云存档，上传玩家的游戏纪录，否则辛辛苦苦肝出来的存档，卸载后就...,1
3,▶前言:专治手欠手残，不知诸位手好了没有？▶评论标题:你的手由我来守护▶简评:①Tap上又一...,1
4,《地下城堡：炼金术师的魔幻之旅》2014.10.1【雷霆游戏】淘金互娱【剧情背景】误入荒野天...,1


In [0]:
df_taptap = pd.concat([df_good_new,df_bad_new], ignore_index=True)

In [0]:
df_taptap_final = df_taptap.sample(frac=1).reset_index(drop=True)

In [0]:
df_taptap_final.sentiment.value_counts()

0    2458
1    2430
Name: sentiment, dtype: int64

In [0]:
df_taptap_final.to_csv(path/'taptap_review_ready.csv',index=False)

## Data Load

In [0]:
tap_df_pro = pd.read_csv(path/'taptap_processed.csv',index_col=0)

In [0]:
tap_df_pro.head()

,Unnamed: 0,start_link-href,comment_texts,stars,sentiment
0,0,https://www.taptap.com/app/78946,看见这个游戏上了日推，于是便心血来潮的的下载玩了一下，现在借助Tap这个平台发一些玩这个游戏...,width: 56px,1
1,1,https://www.taptap.com/app/16404,之前手机玩过一段时间，个人不太喜欢这种类型。很不明白，为什么不做刀剑2，当年端游方面很着迷入...,width: 28px,0
2,2,https://www.taptap.com/app/15386,我想给厂家提个建议：可否出一个云存档，上传玩家的游戏纪录，否则辛辛苦苦肝出来的存档，卸载后就...,width: 56px,1
3,3,https://www.taptap.com/app/53320,▶前言:专治手欠手残，不知诸位手好了没有？▶评论标题:你的手由我来守护▶简评:①Tap上又一...,width: 56px,1
4,4,https://www.taptap.com/app/11471,《地下城堡：炼金术师的魔幻之旅》2014.10.1【雷霆游戏】淘金互娱【剧情背景】误入荒野天...,width: 56px,1


In [0]:
pandas_profiling.ProfileReport(tap_df_pro)

In [0]:
tap_df_pro.to_csv(path/'taptap_processed.csv',index=False)